# Тематическое моделирование

Я честно не придумала, на каком корпусе можно протестировать то, что у меня получится, и решила оставить корпус фантастики. Вместо этого у меня появилась гипотеза для метрики по определению количества топиков.

In [ ]:
# а то забуду
random_seed = 42

## Предобработка

In [ ]:
import os

from nltk.corpus import stopwords
from pymystem3 import Mystem

In [ ]:
def preprocess_text(text, stops, mystem):
    lemmas_list = []
    for word_analysis in mystem.analyze(text):
        if word_analysis.get("analysis"):
            lemma = word_analysis["analysis"][0]["lex"]
            if lemma not is stops:
                lemmas_list.append(lemma)
    text_lemmatized = " ".join(lemmas_list)
    return text_lemmatized

In [ ]:
mystem = Mystem()
stops = stopwords.words("russian")

In [ ]:
path_to_corpus = "../data/fantasy_corpus"
processed_texts = []

for item in [fn for fn in os.listdir(path_to_corpus) if fn.endswith(".txt")]:
    with open(os.path.join(path_to_corpus, item), "r", encoding="utf-8") as f:
        raw_text = f.read()
    processed_texts.append(preprocess_text(raw_text, stops, mystem))

## TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf_vec = TfidfVectorizer()
tfidf_m = tfidf_vec.fit_transform(processed_texts)

## Обучение и выбор числа топиков

**Идея:** я хочу попробовать оттолкнуться от семантической близости слов в топике. Если топик выделился осознанно, то семантическая близость первых 20\* слов будет высокой.

Сейчас (до того, как я начала писать код и проверять эту гипотезу) есть одно «но» — в топик могут входить не только синонимы, но и антонимы (например, слова «дорогой» и «дешёвый» вполне могут вместе попасть в финансовый топик), семантическая близость которых равна -1; тем не менее, кажется, что всё-таки даже если и так, пара антонимов не так сильно утянет значение средней семантической близости вниз, как набор слов в стиле «стул, лошадь, 23».

_\* почему 20? Если мы говорим о «авторитарных» топиках, то кажется, что такое число может отразить и важные слова с большим весом, и менее важные с весом поменьше; в случае с «демократическими» топиками 20 — достаточное число слов, чтобы зацепить разные аспекты, которые могут быть в топике._

### Проверка топиков

Я буду использовать модель с такими параметрами:

* обучена на текстах НКРЯ образца 2019 года,

* 270 миллионов слов, объём словаря — 189 193 слова.

In [ ]:
import gensim

In [ ]:
w2v_model_path = "../data/ruscorpora_upos_cbow_300_20_2019.zip"
with zipfile.ZipFile(w2v_model_path, "r") as archive:
    stream = archive.open("model.bin")
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

Как считается семантическая близость _слов в топике_: 

1. составляются все возможные пары из слов, 

2. для каждой считается сем. близость,

3. берётся среднее значение всех полученных величин.

In [ ]:
from itertools import combinations
from statistics import mean

In [ ]:
def assess_topic(topic_words, model):
    sem_similarities = []
    for words_pair in combinations(topic_words):
        try:
            sim = model.similarity(words_pair)
        except:
            sim = 0
        sem_similarities.append(sim)
    return mean(sem_similarities)

In [ ]:
def assess_trained_lda(all_topic_words, model):
    all_topic_scores = []
    for this_topic_words in all_topic_words:
        this_topic_score = assess_topic(this_topic_words)
        all_topic_scores.append(this_topic_score)
    return all_topic_scores

### Обучение модели

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
def fit_lda(feature_m, num_topics):
    # обучаем модель
    lda = LatentDirichletAllocation(n_components=num_topics,
                                    learning_method="online",
                                    random_state=random_seed)
    lda = lda.fit(feature_m)
    # забираем топ-20 слов топика
    # кусочек кода здесь взят из gist-а Бориса Валерьевича, спасибо!
    num_top_words = 20
    topics = []
    for topic_idx, topic in enumerate(lda.components_):
        topic_words = [feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]
        topics.append(topic_words)
    return lda, topics

Разброс топиков сделаю на порядок — от 5 до 50.

_почему 5? не знаю, просто захотелось 5…_

In [ ]:
experiment_scores = {}
experiment_topics = {}

for topic_num in range(5, 51):
    lda_trained, all_topics_words = fit_lda(feature_m, topic_num)
    all_topics_scores = assess_trained_lda(all_topics_words, model)
    experiment_scores[topic_num] = mean(all_topics_scores)
    experiment_topics[topic_num] = all_topics_words

Все результаты сложу в отдельный файл, чтобы тетрадка не превратилась в портянку:

In [ ]:
with open("./topic-modelling_topics.txt", "w", encoding="utf-8") as file_topics:
    for topic_num in range(5, 51):
        file_topics.write("{}\n{}".format(topic_num, "\n".join(experiment_topics[topic_num])))

with open("./topic-modelling_scores.txt", "w", encoding="utf-8") as file_scores:
    for topic_num in range(5, 51):
        file_scores.write("{}\n{}".format(topic_num, " ".join(experiment_scores[topic_num])))

## Результаты

Посмотрим на результат работы модели с лучшей метрикой:

In [ ]:
best_score_topic_num = max(experiment_scores, key=experiment_scores.get)
best_score_topic_num

In [ ]:
lda_trained, all_topics_words = fit_lda(feature_m, best_score_topic_num)
for i, topic in enumerate(all_topics_words):
    print("topic no: {}\twords: {}".format(i, ", ".join(all_topics_words)))

_Бонус:_ ничто не ново под луной — семантику прикручивали ещё в 2013 ([и даже ещё более сложную](https://www.researchgate.net/publication/235974307_Evaluating_Topic_Coherence_Using_Distributional_Semantics)), но результат был сравнительно приятный.

## Что ещё можно было бы попробовать сделать?

Пока я писала плюсы и минусы своей идеи, мне пришло в голову, что тут хорошо бы справилась какая-нибудь онтология или ворднет — благодаря тому, что в них реализована идея уровневости; можно было бы смотреть, совпадают ли у слов в топиках гиперонимы.